# AMPCs 

## Imports

In [ ]:
import os
import warnings
import numpy as np

# Suppress UserWarnings
# warnings.filterwarnings("ignore", category=UserWarning)

from tqdm.notebook import tqdm_notebook
from pprint import pprint

In [ ]:
from src.parameters import AMPC_Param
from src.generate_trajs import generate_AMPC_trajs
from src.utils import get_all_acados_options

## Settings

In [ ]:
RESULTS_DIR = os.path.abspath('Results')
MPC_RESULTS_DIR = os.path.join(RESULTS_DIR, 'AMPC_results')

MAKE_TEST_RESULTS = False

if MAKE_TEST_RESULTS:
    MPC_RESULTS_DIR += '_Test'

NUM_AMPCS = 10

MPC_PARAM_DICT = {
    'T_sim': 5, # length of the closed-loop simulation (in seconds)
}

HORIZON_OPTIONS = (30, )

In [ ]:
# parameters needed for neural horizon acados MPC
AMPC_PARAMS = [
    AMPC_Param(
        # Param
        N_MPC = N_MPC, 
        N = N_MPC,
        version = VERSION,
 
        # Param
        **MPC_PARAM_DICT
    ) for VERSION in range(NUM_AMPCS) \
        for N_MPC in HORIZON_OPTIONS
]

### Acados options

In [ ]:
# qp_solver
# ---------
#       'FULL_CONDENSING_QPOASES', 'FULL_CONDENSING_HPIPM', 'FULL_CONDENSING_DAQP',
#       'PARTIAL_CONDENSING_HPIPM', 'PARTIAL_CONDENSING_QPDUNES', 'PARTIAL_CONDENSING_OSQP'
# hessian_approx
# --------------
#       'GAUSS_NEWTON', 'EXACT'
# integrator_type
# ---------------
#       'ERK', 'IRK', 'DISCRETE', 'LIFTED_IRK', 'GNSF'
# nlp_solver_type
# ---------------
#       'SQP_RTI', 'SQP'
# regularize_method
# -----------------
#       'NO_REGULARIZE', 'MIRROR', 'PROJECT', 'PROJECT_REDUC_HESS', 'CONVEXIFY'
# hpipm_mode
# ----------
#       'BALANCE', 'SPEED_ABS', 'SPEED', 'ROBUST'
# collocation_type
# ----------------
#       'GAUSS_RADAU_IIA', 'GAUSS_LEGENDRE'
# globalization
# -------------
#       'FIXED_STEP', 'MERIT_BACKTRACKING'
# nlp_solver_tol_stat
# -------------------
#       Default: 1e-6
# nlp_solver_max_iter
# -------------------
#       Default: 100
# as_rti_iter
# -----------
#       Default: 1
# as_rti_level
# ------------
#       Default: 4
# 
#############################################
# use_iter_rti_impl
# -----------------
#       Default = False
# use_initial_guesses
# -----------------
#       Default = False
# max_rti_iters
# -----------------
#       Default = 10
# rti_tol
# -----------------
#       Default = 1e-4


# SOLVER_OPTIONS = get_all_acados_options(base_cname='opts')

# filter_out = 'QPDUNES'
# SOLVER_OPTIONS = {cname: aopts for cname, aopts in SOLVER_OPTIONS.items() if filter_out not in aopts['qp_solver']}

SOLVER_OPTIONS = {
    f'ASRTID_FCH': dict(
        qp_solver='FULL_CONDENSING_HPIPM', 
        integrator_type='DISCRETE', 
        nlp_solver_type='SQP_RTI',
        as_rti_iter=3,
        as_rti_level=3,
        nlp_solver_tol_stat=1e-6,
        nlp_solver_max_iter=3,
    ),
    # f'SQP_FCHPIPM_DISCRETE': dict(
    #     qp_solver='FULL_CONDENSING_HPIPM', 
    #     integrator_type='DISCRETE', 
    #     nlp_solver_type='SQP',
    #     use_initial_guesses=True,
    # ),
}

In [ ]:
with tqdm_notebook(total=len(SOLVER_OPTIONS)*len(AMPC_PARAMS), unit='MPC', desc='Get trajectories: ') as tqdm_handle:
    for ampc_param in AMPC_PARAMS: 
        for acados_option in SOLVER_OPTIONS.items():
            name = f'{acados_option[0]} {ampc_param.N_MPC}M {ampc_param.version}v'
            tqdm_handle.set_description_str(f'Get trajectory of:\n{name}')
            results = generate_AMPC_trajs(ampc_param, acados_option, MPC_RESULTS_DIR)

            tqdm_handle.update(1)
            tqdm_handle.set_postfix_str(f'\n{name}:\nCost:{results.Cost:.2f}, Time:{np.mean(results.Time)*1e3:.2f}ms')